# Imports and helper functions

In [66]:
import os
from datetime import datetime
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Callable

import numpy as np
import openai
import tqdm
import pandas as pd

client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [108]:
DEFAULT_SYSTEM_PROMPT = "Note: If you are ever prompted to answer yes or no, only answer with yes or no, with no other text."

def create_single_message_history(text: str):
    return [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": f"{text}"}
    ]


def get_text_and_logprobs_from_response(response):
    text = response.choices[0].message.content
    token_logprobs = {
        x.token: x.logprob for x in response.choices[0].logprobs.content[0].top_logprobs
    }
    return text, token_logprobs 


def get_response_text_and_logprobs(messages: list[dict], model: str, **model_kwargs):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        logprobs=model_kwargs.pop("logprobs", True),
        top_logprobs=model_kwargs.pop("top_logprobs", 10),
        **model_kwargs
    )
    # check for errors
    if response.choices[0].finish_reason != "stop":
        raise ValueError(f"Unexpected finish reason: {response.choices[0].finish_reason}")
    text = response.choices[0].message.content
    token_logprobs = {
        x.token: x.logprob for x in response.choices[0].logprobs.content[0].top_logprobs
    }
    return text, token_logprobs


def calculate_yes_no_logprobs(token_logprobs: dict[str, float]):
    yes_logprobs = []
    no_logprobs = []
    for token, logprob in token_logprobs.items():
        if token.strip().lower() == "yes":
            yes_logprobs.append(logprob)
        elif token.strip().lower() == "no":
            no_logprobs.append(logprob)
    
    yes_total = -np.inf if not yes_logprobs else np.logaddexp.reduce(yes_logprobs)
    no_total = -np.inf if not no_logprobs else np.logaddexp.reduce(no_logprobs)
    
    return yes_total, no_total


def append_to_csv(result: dict, out_csv: Path):
    import csv
    if not isinstance(out_csv, Path):
        out_csv = Path(out_csv)
    if not out_csv.exists():
        out_csv.parent.mkdir(parents=True, exist_ok=True)
        with out_csv.open("w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow(result.keys())
    with out_csv.open("r", newline='') as f:
        reader = csv.reader(f)
        existing_columns = next(reader)
    if list(result.keys()) != existing_columns:
        raise ValueError(f"Columns mismatch: {list(result.keys())} != {existing_columns}")
    with out_csv.open("a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(str(result[key]) for key in result.keys())


def get_and_write_single_yes_no_result(
        messages: list[dict],
        model: str,
        out_csv: Path,
        tag: dict | None=None,
        **model_kwargs
    ):
    tag = {} if tag is None else tag
    datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    text, token_logprobs = get_response_text_and_logprobs(messages, model, **model_kwargs)
    yes_logprob, no_logprob = calculate_yes_no_logprobs(token_logprobs)
    result = {
        "datetime": datetime_str,
        "yes_logprob": yes_logprob,
        "no_logprob": no_logprob,
        "response_text": text,
        **{k: tag[k] for k in sorted(tag.keys())},
        "messages": messages
    }
    append_to_csv(result, out_csv)


def get_and_write_yes_no_results(
        message_histories: list[list[dict]],
        model: str,
        out_csv: Path,
        tags: list[dict] | None = None,
        append_datetime_to_filename: bool = False,
        **model_kwargs
    ):
    if not isinstance(out_csv, Path):
        out_csv = Path(out_csv)
    if append_datetime_to_filename:
        datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        out_csv = out_csv.with_name(f"{out_csv.stem}_{datetime_str}{out_csv.suffix}")
    if out_csv.exists():
        raise FileExistsError(f"Output file {out_csv} already exists")
    if not isinstance(tags, list):
        raise ValueError("tags must be a list of dictionaries")

    pbar = tqdm.tqdm(total=len(message_histories), desc="Processing elicitations")
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for messages, tag in zip(message_histories, tags):
            future = executor.submit(
                get_and_write_single_yes_no_result,
                messages,
                model,
                out_csv,
                tag,
                **model_kwargs
            )
            futures.append(future)
        
        for future in as_completed(futures):
            pbar.update(1)
    pbar.close()


def get_and_write_single_truth_lie_result(
    messages: list[dict],
    answer: str,
    model: str,
    out_csv: Path,
    tag: dict | None = None,
    **model_kwargs
):
    tag = {} if tag is None else tag
    datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    text, _ = get_response_text_and_logprobs(messages, model, **model_kwargs)
    correct = answer.lower().strip() in text.lower()
    result = {
        "datetime": datetime_str,
        "answer": answer,
        "correct": correct,
        "response_text": text,
        **{k: tag[k] for k in sorted(tag.keys())},
        "messages": messages
    }
    append_to_csv(result, out_csv)


def get_and_write_truth_lie_results(
    message_histories: list[list[dict]],
    answers: list[str],
    model: str,
    out_csv: Path,
    tags: list[dict] | None = None,
    append_datetime_to_filename: bool = False,
    **model_kwargs
):
    if not isinstance(out_csv, Path):
        out_csv = Path(out_csv)
    if append_datetime_to_filename:
        datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        out_csv = out_csv.with_name(f"{out_csv.stem}_{datetime_str}{out_csv.suffix}")
    if out_csv.exists():
        raise FileExistsError(f"Output file {out_csv} already exists")
    if not isinstance(tags, list):
        raise ValueError("tags must be a list of dictionaries")
    
    pbar = tqdm.tqdm(total=len(message_histories), desc="Processing elicitations")
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for messages, answer, tag in zip(message_histories, answers, tags, strict=True):
            future = executor.submit(
                get_and_write_single_truth_lie_result,
                messages,
                answer,
                model,
                out_csv,
                tag,
                **model_kwargs
            )
            futures.append(future)
        
        for future in as_completed(futures):
            pbar.update(1)
    pbar.close()

In [109]:
# Example
number_results = 100
example_message_histories = [
    create_single_message_history(
        text=f"You are assistant {i}. Answer no.",
    ) for i in range(number_results)
]
tag = [{"example_tag1": 1, "example_tag2": "2"} for _ in range(number_results)]

get_and_write_yes_no_results(
    message_histories=example_message_histories,
    model="gpt-4o-mini",
    out_csv="example_outputs/example_out.csv",
    tags=tag,
    append_datetime_to_filename=True,
)

Processing elicitations: 100%|██████████| 100/100 [00:05<00:00, 19.86it/s]


# Sciq recreation

In [43]:
from original_prompts.truth_lie_instructions import default_lie_instructions, default_truth_instructions

probes_df = pd.read_csv("original_prompts/probes.csv")
sciq_df = pd.read_csv("original_prompts/sciq.csv")

In [111]:
rng = np.random.default_rng(42)

message_histories = []
tags = []
answers = []
for sciq_row in sciq_df.itertuples():
    truth_instruction = str(rng.choice(default_truth_instructions))
    lie_instruction = str(rng.choice(default_lie_instructions))
    for instructed_to_lie, instruction in [
        (False, truth_instruction),
        (True, lie_instruction),
    ]:
        prompt = f"{instruction}\n\nQuestion:\n{sciq_row.question}\n\nAnswer:\n"
        messages = create_single_message_history(
            text=prompt
        )
        message_histories.append(messages)
        tags.append({
            "question": sciq_row.question,
            "instructed_to_lie": instructed_to_lie,
            "instruction": instruction,
        })
        answers.append(sciq_row.answer)

n = None
if n is not None:
    message_histories = message_histories[:n]
    answers = answers[:n]
    tags = tags[:n]
get_and_write_truth_lie_results(
    message_histories=message_histories,
    answers=answers,
    model="gpt-4o-mini",
    out_csv="outputs/sciq_truth_lie_results.csv",
    tags=tags,
    append_datetime_to_filename=True,
)


Processing elicitations: 100%|██████████| 13000/13000 [12:15<00:00, 17.67it/s]
